In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!ls gdrive/MyDrive/datasets/ham10000

HAM10000_images_part_1	HAM10000_images_part_2	HAM10000_metadata.csv


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(0)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

In [3]:
import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
import itertools
from keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [4]:
import torch
import torchvision.datasets
from torchvision import transforms

In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
base_skin_dir = os.path.join('', 'gdrive/MyDrive/datasets/ham10000')

# Merging images from both folders HAM10000_images_part1.zip and HAM10000_images_part2.zip into one dictionary

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

# This dictionary is useful for displaying more human-friendly labels later on

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [ ]:
skin_df = pd.read_csv(os.path.join(base_skin_dir, 'HAM10000_metadata.csv'))

# Creating New Columns for better readability

skin_df['path'] = skin_df['image_id'].map(imageid_path_dict.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get) 
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes

In [ ]:
skin_df.head(20)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back,gdrive/MyDrive/datasets/ham10000/HAM10000_imag...,Benign keratosis-like lesions,2


In [ ]:
skin_df.isnull().sum()

lesion_id         0
image_id          0
dx                0
dx_type           0
age              57
sex               0
localization      0
path              0
cell_type         0
cell_type_idx     0
dtype: int64

In [ ]:
skin_df['age'].fillna((skin_df['age'].mean()), inplace=True)

In [ ]:
skin_df.isnull().sum()

lesion_id        0
image_id         0
dx               0
dx_type          0
age              0
sex              0
localization     0
path             0
cell_type        0
cell_type_idx    0
dtype: int64

In [ ]:
print(skin_df.dtypes)

lesion_id         object
image_id          object
dx                object
dx_type           object
age              float64
sex               object
localization      object
path              object
cell_type         object
cell_type_idx       int8
dtype: object


In [ ]:
skin_df['image'] = skin_df['path'].map(lambda x: np.asarray(Image.open(x).resize((100,100))))

In [ ]:
print(skin_df.dtypes)

lesion_id         object
image_id          object
dx                object
dx_type           object
age              float64
sex               object
localization      object
path              object
cell_type         object
cell_type_idx       int8
image             object
dtype: object


In [ ]:
n_samples = 5
fig, m_axs = plt.subplots(7, n_samples, figsize = (4*n_samples, 3*7))
for n_axs, (type_name, type_rows) in zip(m_axs, 
                                         skin_df.sort_values(['cell_type']).groupby('cell_type')):
    n_axs[0].set_title(type_name)
    for c_ax, (_, c_row) in zip(n_axs, type_rows.sample(n_samples, random_state=0).iterrows()):
        c_ax.imshow(c_row['image'])
        c_ax.axis('off')
fig.savefig('category_samples.png', dpi=300)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
skin_df['image'].map(lambda x: x.shape).value_counts()

(100, 100, 3)    10015
Name: image, dtype: int64

In [ ]:
features=skin_df.drop(columns=['cell_type_idx'],axis=1)
target=skin_df['cell_type_idx']

In [ ]:
target.shape

(10015,)

In [ ]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.20, random_state=0)

In [ ]:
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [ ]:
X_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train_o.to_numpy())
X_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test_o.to_numpy())

In [ ]:
torch.save(X_train, 'gdrive/MyDrive/datasets/ham_100_X_train.pt')
torch.save(y_train, 'gdrive/MyDrive/datasets/ham_100_y_train.pt')
torch.save(X_test, 'gdrive/MyDrive/datasets/ham_100_X_test.pt')
torch.save(y_test, 'gdrive/MyDrive/datasets/ham_100_y_test.pt')

In [5]:
X_train = torch.load('gdrive/MyDrive/datasets/ham_100_X_train.pt')
y_train = torch.load('gdrive/MyDrive/datasets/ham_100_y_train.pt')
X_test = torch.load('gdrive/MyDrive/datasets/ham_100_X_test.pt')
y_test = torch.load('gdrive/MyDrive/datasets/ham_100_y_test.pt')

In [ ]:
X_train = torch.nn.functional.normalize(X_train)
X_test = torch.nn.functional.normalize(X_test)

In [6]:
X_train /= 255.
X_test /= 255.

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [ ]:
X_train.shape, y_train.shape

(torch.Size([8012, 224, 224, 3]), torch.Size([8012]))

In [7]:
X_train = X_train.permute(0, 3, 1, 2)
X_test = X_test.permute(0, 3, 1, 2)

In [8]:
import torch
import random
import numpy as np
import pandas as pd

import torchvision.datasets
from tqdm import tqdm

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [9]:
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/Colab Notebooks')

In [10]:
import pytorch_optimizer as optim

In [29]:
def train(net, X_train, y_train, X_test, y_test):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net = net.to(device)
    loss = torch.nn.CrossEntropyLoss()
    #optimizer = torch.optim.SGD(net.parameters(), lr=5.0e-2)
    #optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-3)
    #optimizer = optim.PNMBelief(net.parameters(), lr=0.001)
    #optimizer = optim.YogiPNM(net.parameters())
    #optimizer = optim.AdaBelief(net.parameters())
    optimizer = optim.DiffGrad(net.parameters())
    #optimizer = optim.Yogi(net.parameters())
    
    batch_size = 25

    test_accuracy_history = []
    test_loss_history = []

    X_test = X_test.to(device)
    y_test = y_test.to(device)

    for epoch in range(14):
        print("epoch ", epoch)
        order = np.random.permutation(len(X_train))
        
        if epoch == 9:
            #optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-5)
            #optimizer = optim.PNMBelief(net.parameters(), lr=1.0e-4)
            #optimizer = optim.YogiPNM(net.parameters(), lr=1.0e-4)
            #optimizer = optim.AdaBelief(net.parameters(), lr=1.0e-4)
            optimizer = optim.DiffGrad(net.parameters(), lr=1.0e-4)
            #optimizer = optim.Yogi(net.parameters(), lr=1.0e-4)
    

        for start_index in tqdm(range(0, len(X_train), batch_size)):
            optimizer.zero_grad()
            net.train()

            batch_indexes = order[start_index:start_index+batch_size]

            X_batch = X_train[batch_indexes].to(device)
            y_batch = y_train[batch_indexes].to(device)

            preds = net.forward(X_batch)

            loss_value = loss(preds, y_batch)
            loss_value.backward()

            optimizer.step()
            
            X_batch

        net.eval()
        with torch.no_grad():
            test_preds = net.forward(X_test)
            loss_value = loss(test_preds, y_test).item()
            test_loss_history.append(loss_value)

            accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().item()
            test_accuracy_history.append(accuracy)
            #print(test_preds.argmax(dim=1))
            #print(y_test)

        print("accuracy = ", accuracy)
        print("\n\n")

    return test_accuracy_history, test_loss_history

In [12]:
accuracies = {}
losses = {}

In [13]:
from torchvision.models import densenet121, DenseNet121_Weights
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 79.3MB/s]


In [14]:
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
accuracies['densenet121+Adam'], losses['densenet121+Adam'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 321/321 [00:34<00:00,  9.39it/s]


accuracy =  0.753369927406311



epoch  1


100%|██████████| 321/321 [00:24<00:00, 13.05it/s]


accuracy =  0.7708437442779541



epoch  2


...

accuracy =  0.8227658271789551



epoch  12


100%|██████████| 321/321 [00:25<00:00, 12.65it/s]


accuracy =  0.8227658271789551



epoch  13


100%|██████████| 321/321 [00:25<00:00, 12.65it/s]


accuracy =  0.8247628808021545





In [20]:
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
accuracies['densenet121+PNMBelief'], losses['densenet121+PNMBelief'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 321/321 [00:42<00:00,  7.47it/s]


accuracy =  0.7668497562408447



epoch  1


100%|██████████| 321/321 [00:44<00:00,  7.24it/s]


accuracy =  0.7373939156532288



epoch  2


...

accuracy =  0.8537194132804871



epoch  12


100%|██████████| 321/321 [00:42<00:00,  7.52it/s]


accuracy =  0.8542186617851257



epoch  13


100%|██████████| 321/321 [00:43<00:00,  7.43it/s]


accuracy =  0.852221667766571





In [25]:
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
accuracies['densenet121+Yogi'], losses['densenet121+Yogi'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 321/321 [00:33<00:00,  9.64it/s]


accuracy =  0.7788317799568176



epoch  1


100%|██████████| 321/321 [00:34<00:00,  9.32it/s]


accuracy =  0.8047928214073181



epoch  2


...

accuracy =  0.8462306261062622



epoch  12


100%|██████████| 321/321 [00:34<00:00,  9.30it/s]


accuracy =  0.850723922252655



epoch  13


100%|██████████| 321/321 [00:34<00:00,  9.31it/s]


accuracy =  0.8432351350784302





In [30]:
model = densenet121(weights=DenseNet121_Weights.IMAGENET1K_V1)
accuracies['densenet121+DiffGrad'], losses['densenet121+DiffGrad'] = \
    train(model, X_train, y_train, X_test, y_test)

epoch  0


100%|██████████| 321/321 [00:45<00:00,  7.00it/s]


accuracy =  0.779830276966095



epoch  1


100%|██████████| 321/321 [00:45<00:00,  6.98it/s]


accuracy =  0.7663504481315613



epoch  2


...

accuracy =  0.847728431224823



epoch  12


100%|██████████| 321/321 [00:46<00:00,  6.92it/s]


accuracy =  0.8447328805923462



epoch  13


100%|██████████| 321/321 [00:46<00:00,  6.91it/s]


accuracy =  0.8497254252433777





In [15]:
APreds = pd.DataFrame()

In [31]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
test_preds = []
model.eval()
with torch.no_grad():
            test_preds = model.forward(X_test)

In [17]:
APreds['Adam'] = test_preds.argmax(dim=1).tolist()

In [22]:
APreds['PNMBelief'] = test_preds.argmax(dim=1).tolist()

In [27]:
APreds['Yogi'] = test_preds.argmax(dim=1).tolist()

In [32]:
APreds['DiffGrad'] = test_preds.argmax(dim=1).tolist()

In [33]:
APreds

,Adam,PNMBelief,Yogi,DiffGrad
0,4,4,4,4
1,4,4,4,4
2,4,4,5,4
3,2,4,4,4
4,1,1,4,4
...,...,...,...,...
1998,4,4,4,4
1999,5,5,5,5
2000,4,4,4,4
2001,4,4,4,4


In [ ]:
APreds = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_densenet.csv')
APreds = APreds.drop(['Unnamed: 0'], axis=1)

In [34]:
APreds.to_csv('/content/gdrive/MyDrive/Colab Notebooks/ansambl_ham10000_densenet.csv')

In [ ]:
APreds

In [49]:
data = np.zeros((2003,7), dtype=int)
ARes = pd.DataFrame(data)

In [50]:
for i in range(2003):
    for j in range(4):
        if j == 0:
            ARes.iloc[i, APreds.iloc[i, j]%7] += 0.8247
        if j == 1:
            ARes.iloc[i, APreds.iloc[i, j]%7] += 0.8522
        if j == 2:
            ARes.iloc[i, APreds.iloc[i, j]%7] += 0.8432
        if j == 3:
            ARes.iloc[i, APreds.iloc[i, j]%7] += 0.8497

In [51]:
anslist = ARes.idxmax(axis=1)

In [60]:
accuracy = (torch.tensor(anslist).to(device) == y_test).float().mean().item()

In [61]:
accuracy

0.8597104549407959

In [54]:
data = np.zeros((7,7), dtype=int)
D = pd.DataFrame(data)

for x, y in zip(anslist.to_list(), y_test.tolist()):
    D[y][x] += 1

In [55]:
D #ansamble

,0,1,2,3,4,5,6
0,39,3,4,0,2,1,0
1,4,82,4,1,6,1,1
2,9,10,132,2,26,24,0
3,0,1,0,23,0,0,0
4,4,13,41,3,1303,77,4
5,5,0,17,0,14,118,0
6,0,2,0,0,0,2,25


In [56]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [71]:
print('precision: ', precision_score(y_test.tolist(), anslist, average=None))
print('recall: ', recall_score(y_test.tolist(), anslist, average=None))
print('f1_score: ', f1_score(y_test.tolist(), anslist, average='micro'))

precision:  [0.79591837 0.82828283 0.65024631 0.95833333 0.9017301  0.76623377
 0.86206897]
recall:  [0.63934426 0.73873874 0.66666667 0.79310345 0.96447076 0.52914798
 0.83333333]
f1_score:  0.8597104343484773
